In [1]:
% load_ext autoreload
% autoreload 2

In [22]:
import utils
import train
import cvae

In [4]:
train_iter, val_iter, test, DE, EN = utils.torchtext_extract(d=0)

In [5]:
print(len(train_iter), len(val_iter))

6143 31


In [6]:
model_name = "test"

gpu = True

num_layers = 2
embed_size = 100
hidden_size = 100
latent_size = 50

num_epochs=10

anneal = utils.kl_anneal_linear


In [7]:
model = cvae.CVAE(len(DE.vocab), len(EN.vocab), embed_size, hidden_size, latent_size, num_layers)
if gpu:
    model.cuda()
print("Number of parameters: {}".format(utils.count_parameters(model)))

Number of parameters: 6730189


/home/kevin/.local/lib/python3.5/site-packages/torch/backends/cudnn/__init__.py:84: UserWarning: PyTorch was compiled without cuDNN support. To use cuDNN, rebuild PyTorch making sure the library is visible to the build system.
  "PyTorch was compiled without cuDNN support. To use cuDNN, rebuild "


In [31]:
train.train(model,model_name, train_iter, val_iter, DE, EN, anneal, num_epochs, gpu)

  0%|          | 0/6143 [00:00<?, ?it/s]/home/kevin/.local/lib/python3.5/site-packages/torch/backends/cudnn/__init__.py:84: UserWarning: PyTorch was compiled without cuDNN support. To use cuDNN, rebuild PyTorch making sure the library is visible to the build system.
  "PyTorch was compiled without cuDNN support. To use cuDNN, rebuild "
  0%|          | 6/6143 [00:03<57:26,  1.78it/s]  


KeyboardInterrupt: 

In [ ]:
perp, elbo, nre, kl = utils.eval_vae(model, val_iter, pad, gpu=gpu)
bleu = utils.test_multibleu(model, val_iter, EN, k=1, gpu=gpu)
print(perp, elbo, nre, kl, bleu)